# Tarea8_IDS_Ayudantia9Style

**Nombre(s):** 

**Fecha:** 

## Introducción

Este cuaderno sigue el estilo y la estructura de la *Ayudantía 9* como referencia. Se realiza la **reducción de dimensionalidad** (PCA, t-SNE, UMAP) y la comparación de modelos de clasificación (Regresión Logística) según lo pedido en la Tarea 8.

Se asume que el archivo `customers_Tarea8.csv` está en el mismo directorio que este notebook.

## 1) Cargar librerías y dataset

In [ ]:
# Librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# UMAP se importa en una celda separada con manejo de error en caso de no estar instalado
try:
    import umap
except Exception as e:
    umap = None

# Cargar el dataset
df = pd.read_csv('customers_Tarea8.csv')
print('Dimensiones del dataset:', df.shape)
df.head()

### Observaciones iniciales del dataset

- Verificar tipos de dato, valores nulos, y distribución básica de las variables.


In [ ]:
# Exploración rápida
df.info()
print('\nValores nulos por columna:\n', df.isnull().sum())
df.describe(include='all').T

## 2) Preprocesamiento

Pasos:
1. Separar la variable objetivo `Response` (solo para la parte de modelos la usaremos).  
2. Codificar variables categóricas con `get_dummies` (drop_first=True).  
3. Estandarizar las variables numéricas.


In [ ]:
# Separar target
if 'Response' not in df.columns:
    raise ValueError('El dataset no contiene la columna Response')

X = df.drop(columns=['Response'])
y = df['Response']

# Convertir fechas si existen y extraer información relevante (ejemplo: Dt_Customer -> año, mes)
if 'Dt_Customer' in X.columns:
    try:
        X['Dt_Customer'] = pd.to_datetime(X['Dt_Customer'], errors='coerce')
        X['Customer_reg_year'] = X['Dt_Customer'].dt.year
        X['Customer_reg_month'] = X['Dt_Customer'].dt.month
        X = X.drop(columns=['Dt_Customer'])
    except Exception as e:
        # si falla, mantener la columna original
        pass

# One-hot encoding para categóricas
X = pd.get_dummies(X, drop_first=True)

# Estandarización
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print('Dimensiones después de encoding y escala:', X_scaled.shape)

## 3) Reducción de dimensionalidad (PCA, t-SNE, UMAP)

Para la visualización pedida, reduciremos a 2 dimensiones cada técnica (PCA, t-SNE, UMAP). Además, guardaremos un PCA que capture al menos el 90% de la varianza.

In [ ]:
# 3.1 PCA a 2 dimensiones
pca_2 = PCA(n_components=2, random_state=42)
X_pca2 = pca_2.fit_transform(X_scaled)
print('Explained variance ratios (2 components):', pca_2.explained_variance_ratio_)

# 3.2 PCA que capture 90% de varianza
pca_90 = PCA(n_components=0.90, random_state=42)
X_pca90 = pca_90.fit_transform(X_scaled)
print('Número de componentes para 90% varianza:', pca_90.n_components_)

# 3.3 t-SNE (puede demorar dependiendo del tamaño del dataset)
tsne = TSNE(n_components=2, random_state=42, init='random')
X_tsne = tsne.fit_transform(X_scaled)

# 3.4 UMAP (si está disponible)
if umap is not None:
    umap_model = umap.UMAP(n_components=2, random_state=42)
    X_umap = umap_model.fit_transform(X_scaled)
else:
    X_umap = None

print('Reducciones generadas.')

### Visualizaciones de cada técnica (2D)
Cada gráfico colorea por la variable `Response`.

In [ ]:
def scatter_2d(arr, labels, title, figsize=(8,6), alpha=0.6, s=10):
    plt.figure(figsize=figsize)
    cmap = ListedColormap(['tab:blue','tab:orange'])
    plt.scatter(arr[:,0], arr[:,1], c=labels, cmap=cmap, s=s, alpha=alpha)
    plt.title(title)
    plt.xlabel('Componente 1')
    plt.ylabel('Componente 2')
    plt.colorbar()
    plt.show()

scatter_2d(X_pca2, y, 'PCA - 2 dimensiones')
scatter_2d(X_tsne, y, 't-SNE - 2 dimensiones')
if X_umap is not None:
    scatter_2d(X_umap, y, 'UMAP - 2 dimensiones')
else:
    print('UMAP no está instalado en el entorno. Para usar UMAP instale umap-learn.')

## 4) Modelos de clasificación: Regresión Logística

Entrenaremos y evaluaremos Regresión Logística en 4 escenarios:
1. Sin reducción (todas las features)  
2. Con PCA (componentes que capturan 90% varianza)  
3. Con t-SNE (2D)  
4. Con UMAP (2D)

Usamos `train_test_split` con `random_state=42` y `test_size=0.2` para todos los casos para una comparación coherente.

In [ ]:
# División train/test para el dataset original
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

def entrenar_evaluar(X_tr, X_te, y_tr, y_te, max_iter=2000):
    model = LogisticRegression(max_iter=max_iter)
    model.fit(X_tr, y_tr)
    preds = model.predict(X_te)
    return accuracy_score(y_te, preds), model, preds

# 1) Sin reducción
acc_orig, model_orig, preds_orig = entrenar_evaluar(X_train, X_test, y_train, y_test)
print('Accuracy (sin reducción):', acc_orig)

# 2) Con PCA 90%
X_pca90_train, X_pca90_test, y_pca_train, y_pca_test = train_test_split(X_pca90, y, test_size=0.2, random_state=42)
acc_pca90, model_pca90, preds_pca90 = entrenar_evaluar(X_pca90_train, X_pca90_test, y_pca_train, y_pca_test)
print('Accuracy (PCA 90%):', acc_pca90)

# 3) Con t-SNE (2D)
X_tsne_train, X_tsne_test, y_tsne_train, y_tsne_test = train_test_split(X_tsne, y, test_size=0.2, random_state=42)
acc_tsne, model_tsne, preds_tsne = entrenar_evaluar(X_tsne_train, X_tsne_test, y_tsne_train, y_tsne_test)
print('Accuracy (t-SNE 2D):', acc_tsne)

# 4) Con UMAP (2D) si está disponible
if X_umap is not None:
    X_umap_train, X_umap_test, y_umap_train, y_umap_test = train_test_split(X_umap, y, test_size=0.2, random_state=42)
    acc_umap, model_umap, preds_umap = entrenar_evaluar(X_umap_train, X_umap_test, y_umap_train, y_umap_test)
    print('Accuracy (UMAP 2D):', acc_umap)
else:
    acc_umap = None
    print('UMAP no disponible - se omitió el entrenamiento con UMAP')

## 5) Tabla comparativa de modelos (accuracy)
Se comparan 4 modelos según lo solicitado en la consigna.

In [ ]:
results = pd.DataFrame({
    'Modelo': [
        'Regresión Logística sin reducción',
        'Regresión Logística con PCA (90%)',
        'Regresión Logística con t-SNE (2D)',
        'Regresión Logística con UMAP (2D)'
    ],
    'Accuracy': [acc_orig, acc_pca90, acc_tsne, acc_umap]
})
results

### Selección del mejor modelo
Elegir el que tenga mayor accuracy y justificar por qué puede ser el mejor en este caso.

In [ ]:
# Mejor modelo según accuracy (se ignoran valores None)
best_row = results[results['Accuracy'] == results['Accuracy'].max()].iloc[0]
best_row

## 6) Análisis y conclusiones (sugerencia de redacción)

- **Impacto de la reducción de dimensionalidad:** describir cómo cambia la performance y el tiempo de entrenamiento.  
- **PCA**: recomendable cuando se busca conservar varianza y reducir dimensionalidad de forma lineal.  
- **t-SNE y UMAP**: útiles para visualización y detección de clusters; su uso como preprocesamiento para modelos lineales puede no mejorar accuracy y a veces lo empeora.  
- **¿Vale la pena el esfuerzo computacional?**: PCA suele justificarlo; t-SNE es costoso y su uso está más orientado a visualización. UMAP es más rápido que t-SNE, pero sigue siendo no-lineal.


## 7) Anexo: Prompts de IA usados (si corresponde)

Se debe incluir aquí cualquier prompt de ChatGPT o similar usado para redactar el cuaderno o las conclusiones, según la política de la asignatura.

In [ ]:
# Guardar resultados y modelos opcionalmente
results.to_csv('resultados_tarea8_accuracy.csv', index=False)
print('Resultados guardados en resultados_tarea8_accuracy.csv')

-----
### Nota final

Este notebook está diseñado para **ejecutarse de corrido** en un entorno con las librerías instaladas (`pandas`, `scikit-learn`, `matplotlib`, `umap-learn` opcional). Si UMAP no está instalado, el cuaderno sigue siendo funcional omitiendo las partes de UMAP.

Antes de entregar, renombrar el archivo al formato exigido por la asignatura: `NombreApellido1_NombreApellido2.ipynb`.